<div style="font-size:22pt; line-height:25pt; font-weight:bold; text-align:center;">Creating Chatbots and recognizing entities</div>

In this exercice, we will apply the example of text classification studied previously to the creation of chatbots.

Building an agent capable of handling conversations is usually performed through the following steps:
1. We apply text classification techniques to the utterance typed by the user, in order to detect the **intent** of the user (e.g. "applying for a loan", "purchasing a product from a store", "asking about the weather", etc)
2. In addition to detecting intent, we extract **Named Entities** from the utterance (e.g. location names, currency amounts, etc...)
3. We use the extracted intents and entities to decide how the chatbot should reply

In this exercise, we will use the open source chatbot framework **Rasa** to build the chatbot. It includes the package *Rasa NLU* that performs NLU tasks such as entity extraction and intent classification, as well as *Rasa Core*, that manages the conversational aspects.

# 1. Using Rasa

Rasa runs code asynchronously. The following cell configures the notebook for Rasa use. See [the documentation of Rasa](https://rasa.com/docs/rasa/api/jupyter-notebooks/) for more information.


In [1]:
import nest_asyncio

nest_asyncio.apply()


# Rasa will throw plenty of warnings during execution. This line gets rid of them, for lisibility. 
# Just comment it to keep the warnings if desired
import warnings
warnings.simplefilter('ignore')

Rasa chatbot project folders follow a precise structure to function. Important files to consider are:
1. A **config.yml** file defining the language of the chatbot, the methods used for intent classification, etc...
2. A **domain.yml** file, defining the intents and entities covered by the chatbot, as well as the utterances and actions it will use. Generally, most of the engineering of chatbots goes towards this file.
3. A **nlu** file (Rasa uses json or md files), containing the training samples for intent classification and entity extraction.
4. A **stories** file, contaning the different scenariis of interaction between the user and the bot. This file defines how to associate bot actions and intents/entities.

When starting a new project, Rasa provides a useful function that prepares the folder structure:

In [2]:
from rasa.cli.scaffold import create_initial_project

project = "my_new_chatbot_project"
create_initial_project(project)

# 2. Building our first chatbot

In [4]:
#project location
project = "chatbot_projects/1_simple_clockbot/"

#path to config, domain, nlu and stories files
config = project+"config.yml"
domain = project+"domain.yml"
training_files = project+"data/"

#where to store the models
output = project+"models/"

Our first chatbot is meant to be a simple clockbot (a chatbot that gives the time). Let's go through the structure of the files to perform this task.

## 2.a. config file

```
# Configuration for Rasa NLU.
# https://rasa.com/docs/rasa/nlu/components/
language: fr
pipeline: supervised_embeddings
```

The chatbot is meant to be in french. The pipeline we use is the default Rasa one, **supervised_embeddings**.

In the text classification examples we saw in the previous exercise, we learnt individual vector representations of every word in our corpus, and used these as features for classification. We saw that it causes issues with out-of-vocabulary (OOV) words, and with model storage. The main modification to embeddings proposed by Rasa is to learn intent representations as ell as words'. This means that any input sequence can be judged in terms of similarity with an intent, making it easier and less costly to perform classification. You can read more about this [here](https://medium.com/rasa-blog/supervised-word-vectors-from-scratch-in-rasa-nlu-6daf794efcd8)

## 2.b. domain file

```
intents:
  - greet
  - ask_time

actions:
- utter_greet
- utter_time

templates:
  utter_greet:
  - text: "Bien le bonjour !"
  utter_time:
  - text: "Il est 10h"
```

As introduced earlier, this first chatbot will:
- cover 2 intents (greeting and asking what time it is)
- perform only two possible actions (greet the user back, or give the time)

Both actions performed by the chatbot in this example are single utterances, defined within the file itself. It would be possible to declare more complex actions, ran separately in an "action server" (see [the documentation](https://rasa.com/docs/rasa/core/actions/#actions)). As the objective of this exercise is not to define an action server, but rather to explore machine learning capabilities of chatbots, we propose to simplify the problem by creating a dumb clockbot stuck at 10 o'clock.

## 2.c. stories file

```
## greet path
* greet
  - utter_greet

## time_path_polite
* greet
  - utter_greet
* ask_time
  - utter_time
  - action_restart
```

We define only two main scenarii for interaction with this chatbot :
- the user greets the chatbot, but interacts no further
- the user greets the chatbot, then asks what time it is (the action_restart at the end of this path reinitializes the bot once it has given the time)

Note that we have not defined a scenario where the user would ask the time without greeting the robot first (we will cover this in the next exercise).

## 2.d. nlu file

```
## intent:greet
- salut
- hello
- yo
- slt
- bonjour
- bonsoir

## intent:ask_time
- il est quelle heure ?
- quelle heure est-il
- t'as l'heure stp ?
```

This file contains examples for all the intents that we have defined for the chatbot. Note that we included some examples containing abbreviations, mispellings, etc... Even if NLU models are designed to be robust to misspellings when trained on correctly spelled utterances, it is a good practice to include examples "from the real world" for each intent, and to include them as such (without correcting misspellings).

## 2.e. Training the chatbot


In [5]:
import rasa

#training the model and saving the path where the model is stored
model_path = rasa.train(domain, config, [training_files], output)


Nothing changed. You can use the old model stored at 'C:\Users\DURANTGA\Documents\Pédagogie\MLclass\12 - Natural Language Processing\chatbot_projects\1_simple_clockbot\models\20201130-150104.tar.gz'.


## 2.f. Using the chatbot



In [6]:
from rasa.jupyter import chat
chat(model_path)

The TensorFlow contrib module will not be included in TensorFlow 2.0.
For more information, please see:
  * https://github.com/tensorflow/community/blob/master/rfcs/20180907-contrib-sunset.md
  * https://github.com/tensorflow/addons
  * https://github.com/tensorflow/io (for I/O related ops)
If you depend on functionality not listed there, please file an issue.








INFO:tensorflow:Restoring parameters from C:\Users\DURANTGA\AppData\Local\Temp\1\tmplz3gd8ht\nlu\component_6_EmbeddingIntentClassifier.ckpt


INFO:tensorflow:Restoring parameters from C:\Users\DURANTGA\AppData\Local\Temp\1\tmplz3gd8ht\nlu\component_6_EmbeddingIntentClassifier.ckpt


Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor


Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor


...

Instructions for updating:
If using Keras pass *_constraint arguments to layers.


Instructions for updating:
If using Keras pass *_constraint arguments to layers.


Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where


Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where


Your bot is ready to talk! Type your messages here or send '/stop'.
bonjour
Bien le bonjour !
il est kel heur
Il est 10h
merci
Bien le bonjour !
/stop


## 2.g. Checking the behaviour of the bot classifier

In this section, we will cofirm the resilience of the bot to mistakes during typing.
We first load the model in to an agent.

In [7]:
from rasa.core.agent import Agent
agent=Agent.load(model_path)

INFO:tensorflow:Restoring parameters from C:\Users\DURANTGA\AppData\Local\Temp\1\tmpknowcmfd\nlu\component_6_EmbeddingIntentClassifier.ckpt


INFO:tensorflow:Restoring parameters from C:\Users\DURANTGA\AppData\Local\Temp\1\tmpknowcmfd\nlu\component_6_EmbeddingIntentClassifier.ckpt


Using the following code, we can then parse various messages and see how the nlu classifier ranks them.
We can check that :
- standard messages for the intents (e.g. "salut") get classified correctly, with very high confidence levels
- messages that vary quite extensively from the canonial question used during training can sill get classified correctly, but with lower confidence levels (e.g. using sms-style "kel h?")
- if the messages differ too much from the samples used during training, the confidence level will decrease dramatically. 

**WARNING** : This example can be biased, as we are using only two intents (classes) with very different syntax (greetings are mostly single words, while more complex sentences are used to ask for time) 

In [8]:
await agent.parse_message_using_nlu_interpreter("salut")

{'intent': {'name': 'greet', 'confidence': 0.9957964420318604},
 'entities': [],
 'intent_ranking': [{'name': 'greet', 'confidence': 0.9957964420318604},
  {'name': 'ask_time', 'confidence': 0.004203528631478548}],
 'text': 'salut'}

In [9]:
await agent.parse_message_using_nlu_interpreter("kel h")

{'intent': {'name': 'ask_time', 'confidence': 0.5475245118141174},
 'entities': [],
 'intent_ranking': [{'name': 'ask_time', 'confidence': 0.5475245118141174},
  {'name': 'greet', 'confidence': 0.4524754583835602}],
 'text': 'kel h'}

In [10]:
await agent.parse_message_using_nlu_interpreter("compte")

{'intent': {'name': 'greet', 'confidence': 0.5708716511726379},
 'entities': [],
 'intent_ranking': [{'name': 'greet', 'confidence': 0.5708716511726379},
  {'name': 'ask_time', 'confidence': 0.4291284382343292}],
 'text': 'compte'}

# 3. Adding different paths

In the previous example, we did not consider the case when a user would ask what time it is without greeting the bot first. In this example, we propose to adapt the scenarii of user interaction to handle those cases.

The main modification is located in the **stories** file, where the following story is added :
```
## time_path_impolite
* ask_time
  - utter_impolite
```

Note that an action (utterance) has been added to handle the situation where the bot gets offended because the user did not greet it. This is also reflected in the **domain.yml** file.

In [11]:
#project location
project = "chatbot_projects/2_obnoxious_clockbot/"

#path to config, domain, nlu and stories files
config = project+"config.yml"
domain = project+"domain.yml"
training_files = project+"data/"

#where to store the models
output = project+"models/"

Let's train our bot and run it !

In [12]:
#training the model and saving the path where the model is stored
model_path = rasa.train(domain, config, [training_files], output)

Training Core model...


Processed trackers: 100%|██████████████████████████████████████████████████| 3/3 [00:00<00:00, 100.27it/s, # actions=7]
Processed actions: 7it [00:00, 412.77it/s, # examples=7]
Processed trackers: 100%|██████████████████████████████████████████████| 102/102 [00:02<00:00, 49.59it/s, # actions=49]

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
masking (Masking)            (None, 5, 14)             0         
_________________________________________________________________
lstm (LSTM)                  (None, 32)                6016      
_________________________________________________________________
dense (Dense)                (None, 12)                396       
_________________________________________________________________
activation (Activation)      (None, 12)                0         
Total params: 6,412
Trainable params: 6,412
Non-trainable params: 0
_________________________________________________________________
Train on 49 samples
Epoch 1/100
49/49 [==============================] - 1s 12ms/sample - loss: 2.5733 - acc: 0.0204
Epoch 2/100
49/49 [==============================] - 0s 529us/sample - loss: 2.5304 - acc: 0.0204
Epoch 3/100
49/49 [==================

49/49 [==============================] - 0s 387us/sample - loss: 1.1089 - acc: 0.5510
Epoch 75/100
49/49 [==============================] - 0s 285us/sample - loss: 1.1405 - acc: 0.5918
Epoch 76/100
49/49 [==============================] - 0s 305us/sample - loss: 1.1288 - acc: 0.5918
Epoch 77/100
49/49 [==============================] - 0s 346us/sample - loss: 1.1092 - acc: 0.5714
Epoch 78/100
49/49 [==============================] - 0s 305us/sample - loss: 1.1054 - acc: 0.5714
Epoch 79/100
49/49 [==============================] - 0s 407us/sample - loss: 1.0783 - acc: 0.5918
Epoch 80/100
49/49 [==============================] - 0s 366us/sample - loss: 1.0568 - acc: 0.6122
Epoch 81/100
49/49 [==============================] - 0s 305us/sample - loss: 1.1102 - acc: 0.5714
Epoch 82/100
49/49 [==============================] - 0s 305us/sample - loss: 1.0820 - acc: 0.5918
Epoch 83/100
49/49 [==============================] - 0s 325us/sample - loss: 1.0757 - acc: 0.5918
Epoch 84/100
49/49 [===

Instructions for updating:
Use `tf.compat.v1.data.get_output_types(dataset)`.


...

Instructions for updating:
Use keras.layers.dropout instead.


Instructions for updating:
`tf.batch_gather` is deprecated, please use `tf.gather` with `batch_dims=-1` instead.


Instructions for updating:
`tf.batch_gather` is deprecated, please use `tf.gather` with `batch_dims=-1` instead.



Epochs: 100%|█████████████████████████████████████████████████| 300/300 [00:31<00:00,  9.53it/s, loss=0.515, acc=1.000]


NLU model training completed.
Your Rasa model is trained and saved at 'C:\Users\DURANTGA\Documents\Pédagogie\MLclass\12 - Natural Language Processing\chatbot_projects\2_obnoxious_clockbot\models\20201130-151036.tar.gz'.


The two cells below run the polite scenario (greeting + asking for time) and the impolite one (asking for time directly).
We confirm here that the bot behaves differently in those cases.

In [14]:
agent=Agent.load(model_path)
print(await agent.handle_text("slt"))
print(await agent.handle_text("il est kel h ?"))

INFO:tensorflow:Restoring parameters from C:\Users\DURANTGA\AppData\Local\Temp\1\tmpgfmv035f\nlu\component_6_EmbeddingIntentClassifier.ckpt


INFO:tensorflow:Restoring parameters from C:\Users\DURANTGA\AppData\Local\Temp\1\tmpgfmv035f\nlu\component_6_EmbeddingIntentClassifier.ckpt


[{'recipient_id': 'default', 'text': 'Bien le bonjour !'}]
[{'recipient_id': 'default', 'text': 'Il est 10h'}]


In [15]:
agent=Agent.load(model_path)
print(await agent.handle_text("il est kel h ?"))

INFO:tensorflow:Restoring parameters from C:\Users\DURANTGA\AppData\Local\Temp\1\tmp2d086qyz\nlu\component_6_EmbeddingIntentClassifier.ckpt


INFO:tensorflow:Restoring parameters from C:\Users\DURANTGA\AppData\Local\Temp\1\tmp2d086qyz\nlu\component_6_EmbeddingIntentClassifier.ckpt


[{'recipient_id': 'default', 'text': 'Tu pourrais au moins me dire bonjour !'}]


# 4. Adding entities

In this example, we will add **Named Entity extraction** to the bot. 

Named Entities can be viewed as instances of linguistic classes. For instance, there is an inifinite number of ways through which one might refer to a Location. Some examples could be "Paris", "my house", "Japan", "the restaurant I like on Main Street", etc... From a Named Entity Recognition (NER) perspective, all these examples would be specific **values** (or synonyms) refering to the **entity** "location".

## 4.a How are entities extracted by Rasa ?

### Symbolic approach

The simplest way of approaching Named Entity Extraction is to define all synonyms of an entity manually. In this case, the user would type all possible values of an entity manually. The extraction of these entities would then be performed by matching words in an utterance with the list of synonyms.

There are some cases where this mode of definition is appropriate. For instance, to define the entity "*means of payment*", there are only limited options (*cash*,*cheque*,*credit card*,*debit card*...). This approach, however, has two main issues :
- the maintenance of synonym lists can quickly become impossible (for instance when defining entities corresponding to person or loction names)
- this approach is not resilient to spelling variants (e.g. typing "crdt card" instead of "credit card"). 

The second issue can be mitigated by introducing "flexible" or "fuzzy" matching for entities (i.e. considering that a word that differs by only one character from one of the synonyms is still a match). However, in many cases, this is still imperfect.

### Statistical approach

Rasa relies on **extractors** for entities. For instance, the default CRFExtractor uses Conditional Random Fields fitted on word embeddings to predict whether a given word is an entity. In practice, that means that entities are detected using their representation, as well as the representations of the words in their surrounding.  

In our case, using this approach means that giving enough examples in the form "Quelle heure est-il à Paris", "quelle heure est-il à Londres", etc... will enable the extractor to understand that in the pattern "quelle heure est-il à ....", the word following "à" is a "location".

## 4.b. Defining location entities for our chatbot

We propose to add the capacity to ask what the time is at a given location to our chatbot.

In [16]:
#project location
project = "chatbot_projects/3_obnoxious_clockbot_with_entities/"

#path to config, domain, nlu and stories files
config = project+"config.yml"
domain = project+"domain.yml"
training_files = project+"data/"

#where to store the models
output = project+"models/"

The initial modification is located in the **domain** file, where the entity type "*location*" is defined. We also use the value of this entity in the "*utter_time*" utterance.
```
entities:
  - location

[...]

  utter_time:
  - text: "Il est 10h à {location}"
```

The training data in the **nlu** file is also modified to reflect this change :

```
## intent:ask_time
- il est quelle heure à [New York](location)
- quelle heure est-il à [Paris](location)
- quelle heure est-il à [Séoul](location)
- t'as l'heure de [Tokyo](location) stp ?
- heure de [Lyon](location)
```

In the above examples, the sentence ```Quelle heure est-il à [Lyon](location)``` should be read as "*Quelle heure est-il à Lyon*", but where Lyon is an instance of the "location" entity.
More information about the format of training data can be found in [the documentation](https://rasa.com/docs/rasa/nlu/training-data-format/).

## 4.c. Training and testing the chatbot

In [17]:
#training the model and saving the path where the model is stored
model_path = rasa.train(domain, config, [training_files], output)

Processed Story Blocks: 100%|████████████████████████████████████████████| 3/3 [00:00<00:00, 107.42it/s, # trackers=12]

Training Core model...



Processed trackers: 100%|███████████████████████████████████████████████████| 3/3 [00:00<00:00, 91.15it/s, # actions=7]
Processed actions: 7it [00:00, 233.93it/s, # examples=7]
Processed trackers: 100%|██████████████████████████████████████████████| 110/110 [00:02<00:00, 47.02it/s, # actions=73]


Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
masking (Masking)            (None, 5, 16)             0         
_________________________________________________________________
lstm (LSTM)                  (None, 32)                6272      
_________________________________________________________________
dense (Dense)                (None, 12)                396       
_________________________________________________________________
activation (Activation)      (None, 12)                0         
Total params: 6,668
Trainable params: 6,668
Non-trainable params: 0
_________________________________________________________________
Train on 73 samples
Epoch 1/100
73/73 [==============================] - 1s 8ms/sample - loss: 2.3617 - acc: 0.3562
Epoch 2/100
73/73 [==============================] - 0s 383us/sample - loss: 2.2693 - acc: 0.4658
Epoch 3/100
73/73 [===================

73/73 [==============================] - 0s 287us/sample - loss: 0.8471 - acc: 0.8219
Epoch 75/100
73/73 [==============================] - 0s 328us/sample - loss: 0.8600 - acc: 0.7808
Epoch 76/100
73/73 [==============================] - 0s 273us/sample - loss: 0.8637 - acc: 0.7945
Epoch 77/100
73/73 [==============================] - 0s 369us/sample - loss: 0.8432 - acc: 0.7808
Epoch 78/100
73/73 [==============================] - 0s 369us/sample - loss: 0.8249 - acc: 0.8356
Epoch 79/100
73/73 [==============================] - 0s 369us/sample - loss: 0.8096 - acc: 0.8356
Epoch 80/100
73/73 [==============================] - 0s 355us/sample - loss: 0.8069 - acc: 0.8082
Epoch 81/100
73/73 [==============================] - 0s 424us/sample - loss: 0.8114 - acc: 0.8082
Epoch 82/100
73/73 [==============================] - 0s 492us/sample - loss: 0.7907 - acc: 0.8630
Epoch 83/100
73/73 [==============================] - 0s 369us/sample - loss: 0.7609 - acc: 0.8356
Epoch 84/100
73/73 [===

Epochs: 100%|█████████████████████████████████████████████████| 300/300 [00:33<00:00,  8.98it/s, loss=0.595, acc=1.000]


NLU model training completed.
Your Rasa model is trained and saved at 'C:\Users\DURANTGA\Documents\Pédagogie\MLclass\12 - Natural Language Processing\chatbot_projects\3_obnoxious_clockbot_with_entities\models\20201130-151217.tar.gz'.


We then test the chatbot using a city name that was not present in the training set.

In [18]:
agent=Agent.load(model_path)
print(await agent.handle_text("salut"))
print(await agent.handle_text("il est quelle heure à Toulouse"))

INFO:tensorflow:Restoring parameters from C:\Users\DURANTGA\AppData\Local\Temp\1\tmpgh2yrjig\nlu\component_6_EmbeddingIntentClassifier.ckpt


INFO:tensorflow:Restoring parameters from C:\Users\DURANTGA\AppData\Local\Temp\1\tmpgh2yrjig\nlu\component_6_EmbeddingIntentClassifier.ckpt


[{'recipient_id': 'default', 'text': 'Bien le bonjour !'}]
[{'recipient_id': 'default', 'text': 'Il est 10h à Toulouse'}]


On an individual utterance, we can check that the model has learned to recognize entities based on the sentence structure.

In [19]:
await agent.parse_message_using_nlu_interpreter("kel h Toulouse")

{'intent': {'name': 'ask_time', 'confidence': 0.9700797200202942},
 'entities': [{'start': 6,
   'end': 14,
   'value': 'Toulouse',
   'entity': 'location',
   'confidence': 0.7203844809327085,
   'extractor': 'CRFEntityExtractor'}],
 'intent_ranking': [{'name': 'ask_time', 'confidence': 0.9700797200202942},
  {'name': 'greet', 'confidence': 0.029920265078544617}],
 'text': 'kel h Toulouse'}

However, there is one major limitation : the example above does not work anymore when the city name is not capitalized (toulouse instead of Toulouse). In fact, EntityExtractors traditionnally use word shapes (e.g. patterns of lowercase or uppercase letters) as predictors. In our training data, nearly all city names are capitalized. One way to correct this mistake would be to multiply examples with various capitalizations. 

In [20]:
await agent.parse_message_using_nlu_interpreter("kel h toulouse")

{'intent': {'name': 'ask_time', 'confidence': 0.9700797200202942},
 'entities': [],
 'intent_ranking': [{'name': 'ask_time', 'confidence': 0.9700797200202942},
  {'name': 'greet', 'confidence': 0.029920265078544617}],
 'text': 'kel h toulouse'}